In [ ]:
!pip install spacy pandas
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 92.4 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
!pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ab515bd538b3a58fa51b93342c2b16035e9f2b37fd5e1a845890680d197b9773
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect


In [ ]:
import pandas as pd
import re
from langdetect import detect, DetectorFactory
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
import spacy
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
#Load the JSON file
df = pd.read_json('Luxury_Beauty.json', lines=True)

In [ ]:
df.head()

,overall,vote,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,style,image
0,2,3,True,"06 15, 2010",A1Q6MUU0B2ZDQG,B00004U9V2,D. Poston,"I bought two of these 8.5 fl oz hand cream, an...",dispensers don't work,1276560000,NaN,NaN
1,5,14,True,"01 7, 2010",A3HO2SQDCZIE9S,B00004U9V2,chandra,"Believe me, over the years I have tried many, ...",Best hand cream ever.,1262822400,NaN,NaN
2,5,NaN,True,"04 18, 2018",A2EM03F99X3RJZ,B00004U9V2,Maureen G,Great hand lotion,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
3,5,NaN,True,"04 18, 2018",A3Z74TDRGD0HU,B00004U9V2,Terry K,This is the best for the severely dry skin on ...,Five Stars,1524009600,{'Size:': ' 3.5 oz.'},NaN
4,5,NaN,True,"04 17, 2018",A2UXFNW9RTL4VM,B00004U9V2,Patricia Wood,The best non- oily hand cream ever. It heals o...,I always have a backup ready.,1523923200,{'Size:': ' 3.5 oz.'},NaN


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 574228 entries, 0 to 574627
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   overall         574228 non-null  int64 
 1   vote            103634 non-null  object
 2   verified        574228 non-null  bool  
 3   reviewTime      574228 non-null  object
 4   reviewerID      574228 non-null  object
 5   asin            574228 non-null  object
 6   reviewerName    574197 non-null  object
 7   reviewText      574228 non-null  object
 8   summary         574053 non-null  object
 9   unixReviewTime  574228 non-null  int64 
 10  style           250821 non-null  object
 11  image           7325 non-null    object
dtypes: bool(1), int64(2), object(9)
memory usage: 53.1+ MB


In [ ]:
df.isna().sum()

,0
overall,0
vote,470594
verified,0
reviewTime,0
reviewerID,0
asin,0
reviewerName,31
reviewText,0
summary,175
unixReviewTime,0


In [ ]:
# Download NLTK resources
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
# Fix langdetect randomness
DetectorFactory.seed = 0

# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

#### Preprocessing function

In [ ]:
def preprocess_text(text):
    # Text Cleaning
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    text = re.sub(r'\s+', ' ', text).strip()  # Normalize spaces

    # Lemmatization and stopword removal
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]
    text = ' '.join(words)

    return text

In [ ]:
# 4. Language detection
def is_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [ ]:
# Apply preprocessing and filter English reviews
df['cleaned_review'] = df['reviewText'].apply(preprocess_text)
df = df[df['reviewText'].apply(is_english)]

# Print cleaned reviews
print(df[['reviewText', 'cleaned_review']])

Streaming output truncated to the last 5000 lines.
really like dry shampoo doesnt bunch chemical smell incredible work really well great feel hand hair expensive decided worth every
Obsessed add much volume make hair weird texture lowkey like apply care oily hair gone volume buy
stuff weird sure anything definitely doesnt seem help like dry shampoo spray shake like two product one fine thin hair greasy prone
crunchy brush get dry hair
Amazing product
received sample foam awesome fullsized product like baitandswitch probably wont buy doesnt consistency doesnt smell isnt product
make hair look amazing straighten need hair dryerno flat iron
Used dispenser wont work Waste money
product wonder super kinky frizzy curly hair dont straighten hair use put damp hair wonder
Great product fine thinning hair
WORK NOTHING CAME
received item spray mechanism wasnt working emailed immediately shipped new one question asked really impressed Cant wait try
buying didnt realize use every 5 wash work fine h

### Apply NER to Extract Entities

In [ ]:
import os
from huggingface_hub import InferenceClient
os.environ['HF_TOKEN'] = 'HuggingFace_Token'

In [ ]:
client = InferenceClient(
    provider="auto",
    api_key=os.environ["HF_TOKEN"],
)


In [ ]:
# Load model
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Jean-Baptiste/roberta-large-ner-english")
model = AutoModelForTokenClassification.from_pretrained("Jean-Baptiste/roberta-large-ner-english")

tokenizer_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
# Create NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

# Step 5: Update extract_entities to use transformers pipeline
def extract_entities(review):
    if not review.strip():
        return []
    try:
        # Run NER pipeline
        entities = nlp(review)
        # Filter for ORG and PRODUCT entities
        return [(ent['word'], ent['entity_group']) for ent in entities
                if ent['entity_group'] in ['ORG', 'PRODUCT']]
    except Exception as e:
        print(f"Error processing review: {e}")
        return []

Device set to use cpu


In [ ]:
# Apply NER to processed reviews
df['entities'] = df['cleaned_review'].apply(extract_entities)

KeyboardInterrupt: 

In [ ]:
# Load spaCy model
nlp = spacy.load('en_core_web_sm')

In [ ]:
# Extract entities (PRODUCT, ORG)
def extract_entities(review):
    doc = nlp(review)
    entities = [(ent.text, ent.label_) for ent in doc.ents if ent.label_ in ['ORG', 'PRODUCT']]
    return entities

# Apply NER to cleaned reviews
df['entities'] = df['cleaned_review'].apply(extract_entities)

# Display results
print(df[['cleaned_review', 'entities']])

In [ ]:
# Create a new DataFrame with cleaned_review and entities
new_df = df[['cleaned_review', 'entities']].copy()
new_df.head()

,cleaned_review,entities
0,bought two 85 fl oz hand cream neither dispens...,[]
1,Believe year tried many many different hand cr...,[]
2,Great hand lotion,[]
3,best severely dry skin hand,[]
4,best non oily hand cream ever heals overnight,[]
